In [1]:
import os
import csv
from datetime import datetime
from FileHandler import (
    check_directory, 
    get_all_csv_files, 
    read_paths_from_file, 
    update_tracker_file, 
    get_last_modified_date
)
from SubTables import process_csv_file
from MasterTable import combine_and_organize

try:
    # Read paths from the text file
    file_paths = 'FilePaths.txt'
    input_directory, output_directory_organized, output_directory_master = read_paths_from_file(file_paths)

    # Debug: Print paths to check their values
    print(f"Input Directory: {input_directory}")
    print(f"Output Directory for Organized Files: {output_directory_organized}")
    print(f"Output Directory for Master Table: {output_directory_master}")

    # Check if input_directory exists and is valid
    check_directory(input_directory)

    # Initialize tracker file path
    tracker_file = os.path.join(output_directory_organized, 'tracker.csv')

    # Get all .csv files in the input directory
    all_csv_files = get_all_csv_files(input_directory)
    
    if not all_csv_files:
        print("No .csv files found in the input directory.")
    else:
        os.makedirs(output_directory_organized, exist_ok=True)
        
        # Process all CSV files
        processed_files = []
        for file in all_csv_files:
            file_path = os.path.join(input_directory, file)
            last_modified_date = get_last_modified_date(file_path)

            # Check if file is modified since last processing
            tracker_entry = []
            
            if os.path.exists(tracker_file):
                with open(tracker_file, 'r') as f:
                    reader = list(csv.reader(f))
                    for row in reader:
                        if row[0] == file:
                            tracker_entry = row
                            break
            
            # If tracker entry exists and file has not been modified since last processing
            if tracker_entry and tracker_entry[2] == last_modified_date:
                print(f"No changes detected in '{file}'. Skipping reprocessing.")
                continue
            
            # Process the file and save organized data
            excel_file, success = process_csv_file(file_path, output_directory_organized)
            if success:
                processed_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                update_tracker_file(tracker_file, file, processed_date, last_modified_date)
                print(f"Processed '{file}' and saved to '{excel_file}'.")
                processed_files.append(excel_file)
            else:
                print(f"Processing failed for '{file}'.")

        # Combine organized data into master table
        os.makedirs(output_directory_master, exist_ok=True)
        combine_and_organize(output_directory_organized, output_directory_master)

except PermissionError as pe:
    print(f"Permission error occurred: {pe}. Please check permissions.")
except FileNotFoundError as fnf:
    print(f"File not found error: {fnf}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Input Directory: N:\clinical\175\Dose escalation - combined analysis\MSD_Panel_Proinflam
Output Directory for Organized Files: C:\Users\IshanVallamsetty\OneDrive - Neogene Therapeutics, Inc\Desktop\ResultsMSDPRO
Output Directory for Master Table: C:\Users\IshanVallamsetty\OneDrive - Neogene Therapeutics, Inc\Desktop\ResultsMSDPRO
Excel file 'C:\Users\IshanVallamsetty\OneDrive - Neogene Therapeutics, Inc\Desktop\ResultsMSDPRO\20231108 NT175 Proinflammatory Panel 1 201-0205-001MSDProinflam_organized_data.xlsx' created successfully with assay results organized and formatted.
Processed '20231108 NT175 Proinflammatory Panel 1 201-0205-001.csv' and saved to 'C:\Users\IshanVallamsetty\OneDrive - Neogene Therapeutics, Inc\Desktop\ResultsMSDPRO\20231108 NT175 Proinflammatory Panel 1 201-0205-001MSDProinflam_organized_data.xlsx'.
No changes detected in '20231121 201-0201-001 d-28 to d14 and 201-0205-001 w6.csv'. Skipping reprocessing.
No changes detected in '20231129 201-0201-001 d28 201-0201-00